In [1]:
cd C:\Users\Sorin\Desktop\COMP 551\Assignments

C:\Users\Sorin\Desktop\COMP 551\Assignments


In [2]:
#!/usr/bin/env python

# Naive Bayes Classifier for Gaussian Distribution input data
# TODO: add reference in report to GitHub link

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import t


class NaiveBayesClassifier:
  def trainClassifier(self, X, Y):
    self.pyy = []
    self.tinfo = []
    N, D = X.shape
    for c in (0, 1):
      pyy_c = (1.0 + np.sum(Y == c)) / (N + 1.0 + 1.0)
      self.pyy.append(pyy_c)

      Xc = X[Y == c]
      tinfo_c = []
      for d in xrange(D):
        xbar = Xc[:,d].mean()
        mu = N*xbar / (1.0 + N)
        precision = 1.0 + N
        alpha = 1.0 + N/2.0
        beta = 1.0 + 0.5*Xc[:,d].var()*N + 0.5*N*(xbar*xbar)/precision

        tinfo_cd = {
          'df': 2*alpha,
          'center': mu,
          'scale': np.sqrt( beta*(precision + 1)/(alpha * precision) ),
        }
        tinfo_c.append(tinfo_cd)
      self.tinfo.append(tinfo_c)

  def predictProbabilities(self, X):
    N, D = X.shape

    posteriors = np.zeros((N, 2))
    for c in (0, 1):
      probability_matrix = np.zeros((N, D))
      for d in xrange(D):
        tinfo_cd = self.tinfo[c][d]
        pdf_d = t.pdf(X[:,d], df=tinfo_cd['df'], loc=tinfo_cd['center'], scale=tinfo_cd['scale'])
        probability_matrix[:,d] = pdf_d
      posteriors_c = np.prod(probability_matrix, axis=1)*self.pyy[c]
      posteriors[:,c] = posteriors_c
    P = posteriors[:,1] / np.sum(posteriors, axis=1)
    return np.round(P)

  def scorePrediction(self, X, Y):
    return np.mean(self.predictProbabilities(X) == Y)

  def createConfusionMatrix(self, X, Y):
    P = self.predictProbabilities(X)
    M = np.zeros((2, 2))
    M[0,0] = np.sum(P[Y == 0] == Y[Y == 0])
    M[0,1] = np.sum(P[Y == 0] != Y[Y == 0])
    M[1,0] = np.sum(P[Y == 1] != Y[Y == 1])
    M[1,1] = np.sum(P[Y == 1] == Y[Y == 1])
    return M

  def calculatePredictionAccuracy(y1,y):
    y = np.ravel(y)
    acc = float(100*(y1==y).sum())/ float(y.shape[0])
    return acc

  def plotROCCurve(y1,y2):
    y2 = np.ravel(y2)
    from sklearn.metrics import roc_curve,auc
    fp,tp,thersh = roc_curve(y1,y2)
    curve_area=auc(fp,tp)
    plt.title('ROC Curve')
    plt.plot(fp,tp,'b', label = 'Area Under Curve = %0.2f'% curve_area)
    plt.legend(loc = 'lower right')
    plt.plot([0,1],[0,1],'r--')
    plt.xlim([-0.1,1.2])
    plt.ylim([-0.1,1.2])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    
    
if __name__ == '__main__':

  # read in data
  dataset_train = pd.read_csv('train.csv')
  dataset_test = pd.read_csv('test.csv')
  dataset_forecast = pd.read_csv('forecast.csv')

  Xtrain = dataset_train.iloc[:, [1,2,3,4]].values
  Ytrain = dataset_train.iloc[:, [5]].values.ravel()

  Xtest = dataset_test.iloc[:, [1,2,3,4]].values
  Ytest = dataset_test.iloc[:, [5]].values.ravel()

  Xforecast = dataset_forecast.iloc[:, [1,2,3,4]].values
  Nforecast = dataset_forecast.iloc[:, [5]].values.ravel()

  # train model
  model = NaiveBayesClassifier()
  model.trainClassifier(Xtrain, Ytrain)

  # test model accuracy
  print "train accuracy:", model.scorePrediction(Xtrain, Ytrain)
  print "test accuracy:", model.scorePrediction(Xtest, Ytest)
  

  # generate confusion matrix
  M = model.createConfusionMatrix(Xtest, Ytest)
  print "confusion matrix:"
  print M
  print "N:", len(Ytest)
  print "sum(M):", M.sum()

  # forecast 2017 attendance
  forecast = model.predictProbabilities(Xforecast)
  attendees = np.column_stack((Nforecast, forecast))
    
  # save 2017 forecast to file
  attendees_df = pd.DataFrame(data=attendees, columns=['Name', 'Attends Miami 2017'])
  attendees_df.to_csv("NaiveBayesForecastMiami2017.csv")

In [10]:
#calculatePredictionAccuracy(Ytest, Y_forecast)